In [ ]:
"""
LINCS 2020 统一数据过滤管道
支持checkpoint缓存，避免重复计算
"""

import numpy as np
import pandas as pd
import h5py
from pathlib import Path
import pickle
import gzip
import shutil
import glob
import json
import hashlib
from datetime import datetime
from scipy.spatial.distance import cosine, pdist, squareform
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity
from numba import jit, prange
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
from typing import Dict, Tuple, Optional, List, Any
import warnings
import gc
from tqdm import tqdm
import logging
warnings.filterwarnings('ignore')

# RDKit imports
try:
    from rdkit import Chem
    from rdkit.Chem import Descriptors, AllChem, FilterCatalog
    from rdkit.Chem.FilterCatalog import FilterCatalogParams
    RDKIT_AVAILABLE = True
    # 禁用RDKit的警告输出
    from rdkit import RDLogger
    RDLogger.DisableLog('rdApp.*')
except ImportError:
    print("⚠️  RDKit not installed. Chemical filters will be unavailable.")
    RDKIT_AVAILABLE = False


class FilterCheckpointManager:
    """
    过滤器checkpoint管理器
    负责保存和加载每个过滤步骤的中间结果
    """
    
    def __init__(self, cache_dir: str, overwrite: bool = True):
        """
        初始化checkpoint管理器
        
        参数:
            cache_dir: checkpoint缓存目录
            overwrite: 是否覆盖已有的checkpoint（默认True）
        """
        self.cache_dir = Path(cache_dir)
        self.cache_dir.mkdir(parents=True, exist_ok=True)
        self.overwrite = overwrite
        self.metadata_file = self.cache_dir / "filter_metadata.json"
        self.metadata = self._load_metadata()
        
        # 配置日志
        self.logger = self._setup_logger()
        
    def _setup_logger(self):
        """配置日志系统"""
        logger = logging.getLogger('FilterPipeline')
        logger.setLevel(logging.INFO)
        
        # 清除已有的handlers
        logger.handlers = []
        
        # 控制台handler
        console_handler = logging.StreamHandler()
        console_handler.setLevel(logging.INFO)
        formatter = logging.Formatter('%(message)s')
        console_handler.setFormatter(formatter)
        logger.addHandler(console_handler)
        
        # 文件handler
        log_file = self.cache_dir / "filter_pipeline.log"
        file_handler = logging.FileHandler(log_file, mode='a', encoding='utf-8')
        file_handler.setLevel(logging.DEBUG)
        detailed_formatter = logging.Formatter(
            '[%(asctime)s] %(levelname)s: %(message)s',
            datefmt='%Y-%m-%d %H:%M:%S'
        )
        file_handler.setFormatter(detailed_formatter)
        logger.addHandler(file_handler)
        
        return logger
    
    def _load_metadata(self) -> Dict:
        """加载元数据"""
        if self.metadata_file.exists():
            try:
                with open(self.metadata_file, 'r', encoding='utf-8') as f:
                    return json.load(f)
            except (json.JSONDecodeError, ValueError) as e:
                # JSON文件损坏，备份后创建新的
                backup_file = self.metadata_file.with_suffix('.json.backup')
                try:
                    shutil.copy(self.metadata_file, backup_file)
                    print(f"⚠️  Metadata file corrupted, backed up to: {backup_file.name}")
                except:
                    pass
                
                # 删除损坏的文件
                try:
                    self.metadata_file.unlink()
                    print(f"🗑️  Removed corrupted metadata file")
                except:
                    pass
                
                print(f"✓ Starting with fresh metadata")
                return {}
            except Exception as e:
                print(f"⚠️  Error loading metadata: {e}")
                return {}
        return {}   
    
    def _convert_to_json_serializable(self, obj):
        """递归转换对象为 JSON 可序列化类型"""
        if isinstance(obj, dict):
            return {k: self._convert_to_json_serializable(v) for k, v in obj.items()}
        elif isinstance(obj, (list, tuple)):
            return [self._convert_to_json_serializable(item) for item in obj]
        elif isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif pd.isna(obj):
            return None
        else:
            return obj

    def _save_metadata(self):
        """保存元数据"""
        # 转换后再保存
        serializable_metadata = self._convert_to_json_serializable(self.metadata)
        with open(self.metadata_file, 'w', encoding='utf-8') as f:
            json.dump(serializable_metadata, f, indent=2, ensure_ascii=False)
    
    def _get_checkpoint_path(self, filter_name: str, filter_params: Dict) -> Path:
        """
        生成checkpoint文件路径
        
        参数:
            filter_name: 过滤器名称
            filter_params: 过滤器参数（用于生成唯一文件名）
        
        返回:
            checkpoint文件路径
        """
        # 生成参数hash
        params_str = json.dumps(filter_params, sort_keys=True)
        params_hash = hashlib.md5(params_str.encode()).hexdigest()[:8]
        
        # 生成可读的参数字符串
        param_parts = []
        for k, v in sorted(filter_params.items()):
            if isinstance(v, (list, tuple)):
                v_str = '_'.join(map(str, v))
            else:
                v_str = str(v)
            param_parts.append(f"{k}={v_str}")
        
        params_readable = '__'.join(param_parts)
        # 限制文件名长度
        if len(params_readable) > 100:
            params_readable = params_readable[:100]
        
        filename = f"{filter_name}__{params_readable}__{params_hash}.pkl"
        return self.cache_dir / filename
    
    def has_checkpoint(self, filter_name: str, filter_params: Dict) -> bool:
        """检查是否存在checkpoint"""
        ckpt_path = self._get_checkpoint_path(filter_name, filter_params)
        return ckpt_path.exists() and not self.overwrite
    
    def load_checkpoint(self, filter_name: str, filter_params: Dict) -> Optional[Dict]:
        """
        加载checkpoint
        
        返回:
            如果成功加载返回数据字典，否则返回None
        """
        ckpt_path = self._get_checkpoint_path(filter_name, filter_params)
        
        if not ckpt_path.exists():
            return None
        
        if self.overwrite:
            self.logger.info(f"   ⚠️  Overwrite mode enabled, ignoring existing checkpoint")
            return None
        
        self.logger.info(f"   ✓ Loading checkpoint: {ckpt_path.name}")
        
        try:
            with open(ckpt_path, 'rb') as f:
                data = pickle.load(f)
            
            # 验证数据完整性
            if 'matrix' not in data or 'metadata' not in data:
                self.logger.warning(f"   ⚠️  Checkpoint corrupted, will recompute")
                return None
            
            self.logger.info(f"   ✓ Loaded: {len(data['matrix']):,} samples")
            return data
            
        except Exception as e:
            self.logger.error(f"   ❌ Error loading checkpoint: {e}")
            return None
    
    def save_checkpoint(
        self, 
        filter_name: str, 
        filter_params: Dict, 
        data: Dict,
        filter_stats: Dict
    ):
        """
        保存checkpoint
        
        参数:
            filter_name: 过滤器名称
            filter_params: 过滤器参数
            data: 数据字典（包含matrix和metadata）
            filter_stats: 过滤统计信息
        """
        ckpt_path = self._get_checkpoint_path(filter_name, filter_params)
        
        # 如果overwrite模式开启，删除旧的checkpoint
        if self.overwrite and ckpt_path.exists():
            self.logger.info(f"   🗑️  Removing old checkpoint: {ckpt_path.name}")
            ckpt_path.unlink()
        
        self.logger.info(f"   💾 Saving checkpoint: {ckpt_path.name}")
        
        # 保存数据
        with open(ckpt_path, 'wb') as f:
            pickle.dump(data, f, protocol=4)
        
        file_size_mb = ckpt_path.stat().st_size / (1024**2)
        self.logger.info(f"   ✓ Saved successfully ({file_size_mb:.1f} MB)")
        
        # 更新元数据
        filter_id = f"{filter_name}__{hashlib.md5(json.dumps(filter_params, sort_keys=True).encode()).hexdigest()[:8]}"
        
        self.metadata[filter_id] = {
            'filter_name': filter_name,
            'filter_params': filter_params,
            'checkpoint_path': str(ckpt_path),
            'timestamp': datetime.now().isoformat(),
            'stats': filter_stats,
            'n_samples': len(data['matrix']),
            'n_compounds': data['metadata']['pert_id'].nunique() if 'pert_id' in data['metadata'].columns else 0
        }
        
        self._save_metadata()
    
    def clear_cache(self, filter_name: Optional[str] = None):
        """
        清除缓存
        
        参数:
            filter_name: 如果指定，只清除该过滤器的缓存；否则清除所有
        """
        if filter_name is None:
            # 清除所有checkpoint
            for ckpt_file in self.cache_dir.glob("*.pkl"):
                ckpt_file.unlink()
            self.metadata = {}
            self._save_metadata()
            self.logger.info(f"   ✓ Cleared all checkpoints")
        else:
            # 清除指定过滤器的checkpoint
            pattern = f"{filter_name}__*.pkl"
            removed_count = 0
            for ckpt_file in self.cache_dir.glob(pattern):
                ckpt_file.unlink()
                removed_count += 1
            
            # 更新元数据
            keys_to_remove = [k for k in self.metadata if self.metadata[k]['filter_name'] == filter_name]
            for key in keys_to_remove:
                del self.metadata[key]
            self._save_metadata()
            
            self.logger.info(f"   ✓ Cleared {removed_count} checkpoints for filter '{filter_name}'")
    
    def get_filter_history(self) -> pd.DataFrame:
        """
        获取过滤历史记录
        
        返回:
            DataFrame格式的过滤历史
        """
        if not self.metadata:
            return pd.DataFrame()
        
        records = []
        for filter_id, info in self.metadata.items():
            record = {
                'filter_name': info['filter_name'],
                'timestamp': info['timestamp'],
                'n_samples': info['n_samples'],
                'n_compounds': info['n_compounds'],
                'params': str(info['filter_params'])
            }
            records.append(record)
        
        df = pd.DataFrame(records)
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        return df.sort_values('timestamp', ascending=False)


class LINCS2020FilterPipeline:
    """
    LINCS 2020统一过滤管道
    整合所有数据质量控制和化学过滤步骤
    """
    
    def __init__(
        self, 
        data_dir: str,
        cache_dir: Optional[str] = None,
        overwrite: bool = True
    ):
        """
        初始化过滤管道
        
        参数:
            data_dir: LINCS 2020数据目录
            cache_dir: checkpoint缓存目录（默认为data_dir/filter_cache）
            overwrite: 是否覆盖已有checkpoint（默认True）
        """
        self.data_dir = Path(data_dir)
        
        if cache_dir is None:
            cache_dir = self.data_dir / "filter_cache"
        
        self.checkpoint_mgr = FilterCheckpointManager(cache_dir, overwrite=overwrite)
        self.logger = self.checkpoint_mgr.logger
        
        # 数据存储
        self.gene_info = None
        self.cell_info = None
        self.compound_info = None
        self.inst_info = None
        self.signatures = None
        self.landmark_col_indices = None
        self.landmark_gene_ids = None
        self.landmark_gene_symbols = None
        
        # 化学过滤器
        self.covalent_smarts = None
        self.brenk_catalog = None
        self.nibr_catalog = None
        
        self._print_header()
    
    def _print_header(self):
        """打印欢迎信息"""
        self.logger.info(f"{'='*80}")
        self.logger.info(f"🔬 LINCS 2020 Unified Filter Pipeline")
        self.logger.info(f"{'='*80}")
        self.logger.info(f"Data directory: {self.data_dir}")
        self.logger.info(f"Cache directory: {self.checkpoint_mgr.cache_dir}")
        self.logger.info(f"Overwrite mode: {'ON' if self.checkpoint_mgr.overwrite else 'OFF'}")
        self.logger.info(f"{'='*80}\n")
    
    # ========== 数据加载方法 ==========
    
    def load_gene_info(self):
        """加载基因信息"""
        gene_file = self.data_dir / "geneinfo_beta.txt"
        
        if not gene_file.exists():
            raise FileNotFoundError(f"Gene info file not found: {gene_file}")
        
        self.logger.info(f"📖 Loading gene information...")
        gene_info = pd.read_csv(gene_file, sep='\t')
        self.logger.info(f"   ✓ Loaded {len(gene_info):,} genes")
        
        if 'feature_space' in gene_info.columns:
            landmark_mask = gene_info['feature_space'] == 'landmark'
            landmark_genes = gene_info[landmark_mask].copy()
            self.landmark_col_indices = np.where(landmark_mask.values)[0]
            self.landmark_gene_ids = set(landmark_genes['gene_id'].astype(str).values)
            self.landmark_gene_symbols = set(landmark_genes['gene_symbol'].astype(str).values)
            self.logger.info(f"   ✓ Landmark genes: {len(landmark_genes):,}")
        else:
            raise ValueError("Cannot identify landmark genes. 'feature_space' column not found.")
        
        self.gene_info = gene_info
        return gene_info
    
    def load_cell_info(self):
        """加载细胞系信息"""
        cell_file = self.data_dir / "cellinfo_beta.txt"
        
        if not cell_file.exists():
            raise FileNotFoundError(f"Cell info file not found: {cell_file}")
        
        self.logger.info(f"📖 Loading cell information...")
        cell_info = pd.read_csv(cell_file, sep='\t')
        self.logger.info(f"   ✓ Loaded {len(cell_info):,} cell lines")
        
        self.cell_info = cell_info
        return cell_info
    
    def load_compound_info(self):
        """加载化合物信息"""
        compound_file = self.data_dir / "compoundinfo_beta.txt"
        
        if not compound_file.exists():
            self.logger.warning(f"⚠️  Compound info file not found: {compound_file}")
            return None
        
        self.logger.info(f"📖 Loading compound information...")
        compound_info = pd.read_csv(compound_file, sep='\t')
        self.logger.info(f"   ✓ Loaded {len(compound_info):,} compounds")
        
        self.compound_info = compound_info
        return compound_info
    
    def load_instance_info(self):
        """加载实例信息"""
        inst_file = self.data_dir / "instinfo_beta.txt"
        
        if not inst_file.exists():
            raise FileNotFoundError(f"Instance info file not found: {inst_file}")
        
        self.logger.info(f"📖 Loading instance information...")
        inst_info = pd.read_csv(inst_file, sep='\t')
        self.logger.info(f"   ✓ Loaded {len(inst_info):,} instances")
        
        if 'inst_id' in inst_info.columns:
            self.instance_join_col = 'inst_id'
        elif 'sample_id' in inst_info.columns:
            self.instance_join_col = 'sample_id'
        else:
            raise ValueError("Neither 'inst_id' nor 'sample_id' found in instinfo")
        
        self.inst_info = inst_info
        return inst_info
    
    def decompress_gctx_file(self, gctx_file):
        """解压GCTX文件"""
        gctx_file = Path(gctx_file)
        
        if not gctx_file.exists():
            raise FileNotFoundError(f"GCTX file not found: {gctx_file}")
        
        if str(gctx_file).endswith('.gz'):
            self.logger.info(f"⚠️  Detected compressed GCTX file, decompressing...")
            
            decompressed_dir = self.data_dir / "_decompressed"
            decompressed_dir.mkdir(exist_ok=True)
            
            output_path = decompressed_dir / gctx_file.stem
            
            if output_path.exists():
                self.logger.info(f"✓ Found existing decompressed file")
                return str(output_path)
            
            with gzip.open(gctx_file, 'rb') as f_in:
                with open(output_path, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out, length=8*1024*1024)
            
            self.logger.info(f"✓ Decompressed to: {output_path}")
            return str(output_path)
        
        return str(gctx_file)
    
    def read_gctx(self, gctx_file, use_landmark_only=True):
        """读取GCTX文件"""
        gctx_file = str(gctx_file)
        self.logger.info(f"📖 Reading GCTX file: {Path(gctx_file).name}")
        
        file_size_gb = Path(gctx_file).stat().st_size / (1024**3)
        self.logger.info(f"   File size: {file_size_gb:.2f} GB")
        
        gctx_file = self.decompress_gctx_file(gctx_file)
        
        with h5py.File(gctx_file, 'r') as f:
            # 确定路径
            if '/0/DATA/0/matrix' in f:
                matrix_dataset = f['/0/DATA/0/matrix']
                row_path = '/0/META/ROW'
                col_path = '/0/META/COL'
            elif '/matrix' in f:
                matrix_dataset = f['/matrix']
                row_path = '/row'
                col_path = '/col'
            else:
                raise ValueError("Cannot find matrix in GCTX file")
            
            matrix_shape = matrix_dataset.shape
            self.logger.info(f"   Matrix shape: {matrix_shape}")
            
            # 读取元数据
            sample_meta = {}
            for key in f[row_path].keys():
                data = f[f'{row_path}/{key}'][:]
                if data.dtype.kind in ['S', 'O', 'U']:
                    try:
                        sample_meta[key] = np.char.decode(data.astype('S'), 'utf-8')
                    except:
                        sample_meta[key] = data.astype(str)
                else:
                    sample_meta[key] = data
            
            gene_meta = {}
            for key in f[col_path].keys():
                data = f[f'{col_path}/{key}'][:]
                if data.dtype.kind in ['S', 'O', 'U']:
                    try:
                        gene_meta[key] = np.char.decode(data.astype('S'), 'utf-8')
                    except:
                        gene_meta[key] = data.astype(str)
                else:
                    gene_meta[key] = data
            
            sample_df = pd.DataFrame(sample_meta)
            gene_df = pd.DataFrame(gene_meta)
            
            # 自动检测并交换ROW/COL
            if self.gene_info is not None:
                n_features_expected = len(self.gene_info)
                if len(sample_df) == n_features_expected and len(gene_df) != n_features_expected:
                    self.logger.info(f"   ⚠️  Detected ROW/COL swap, correcting...")
                    sample_df, gene_df = gene_df, sample_df
            
            # 确定landmark基因的列索引
            landmark_col_indices = None
            if use_landmark_only:
                if self.gene_info is None or not hasattr(self, "landmark_col_indices"):
                    self.load_gene_info()
                
                landmark_col_indices = np.array(self.landmark_col_indices, dtype=int)
                self.logger.info(f"   Using {len(landmark_col_indices)} landmark genes")
            
            # 分块读取
            self.logger.info(f"   Loading data...")
            if landmark_col_indices is not None:
                chunk_size = 610000
                chunks = []
                
                for start_idx in range(0, matrix_shape[0], chunk_size):
                    end_idx = min(start_idx + chunk_size, matrix_shape[0])
                    chunk = matrix_dataset[start_idx:end_idx, landmark_col_indices].astype(np.float32)
                    chunks.append(chunk)
                    
                    if len(chunks) >= 10:
                        merged = np.vstack(chunks)
                        chunks = [merged]
                        gc.collect()
                
                matrix = np.vstack(chunks) if len(chunks) > 1 else chunks[0]
                del chunks
                gc.collect()
            else:
                matrix = matrix_dataset[:].astype(np.float32)
            
            self.logger.info(f"   ✓ Loaded matrix: {matrix.shape}")
            self.logger.info(f"   ✓ Memory: {matrix.nbytes / (1024**3):.2f} GB")
        
        return matrix, sample_df, gene_df
    
    def load_level4_signatures(self):
        """加载Level 4数据"""
        level4_file = self.data_dir / "level4_beta_trt_cp_n1805898x12328.gctx"
        
        if not level4_file.exists():
            pattern = self.data_dir / "level4_beta_trt_cp*.gctx"
            files = glob.glob(str(pattern))
            if not files:
                raise FileNotFoundError(f"Level 4 file not found: {level4_file}")
            level4_file = files[0]
        
        self.logger.info(f"\n{'='*80}")
        self.logger.info(f"📖 Loading Level 4 Signatures")
        self.logger.info(f"{'='*80}")
        
        matrix, sample_meta, gene_meta = self.read_gctx(level4_file, use_landmark_only=True)
        
        # 合并元数据
        if 'id' in sample_meta.columns:
            sample_meta = sample_meta.rename(columns={'id': 'sample_id'})
        elif 'sample_id' not in sample_meta.columns:
            raise ValueError("Cannot find 'id' or 'sample_id' in GCTX metadata")
        
        if self.inst_info is None:
            self.load_instance_info()
        
        join_col = getattr(self, "instance_join_col", 'sample_id')
        merged_meta = sample_meta.merge(self.inst_info, on=join_col, how='left')
        
        if 'pert_id' not in merged_meta.columns:
            raise ValueError("'pert_id' missing after merge")
        
        self.signatures = {
            'matrix': matrix,
            'row_meta': merged_meta,
            'col_meta': gene_meta
        }
        
        return matrix, merged_meta, gene_meta
    
    # ========== 辅助方法 ==========
    @staticmethod
    @jit(nopython=True, parallel=True, fastmath=True)
    def _compute_max_similarities_jit(data, indices, n_total):
        """Numba加速的余弦相似度计算"""
        n = len(indices)
        result = np.full(n_total, -np.inf, dtype=np.float32)
        
        for i in prange(n):
            idx_i = indices[i]
            vec_i = data[i]
            max_sim = -np.inf
            
            for j in range(n):
                if i != j:
                    sim = np.dot(vec_i, data[j])
                    if sim > max_sim:
                        max_sim = sim
            
            result[idx_i] = max_sim
        
        return result
    
    def calculate_cosine_similarity(self, matrix: np.ndarray, pert_ids: pd.Series) -> np.ndarray:
        """计算每个样本与其最近同化合物复制品的余弦相似度"""
        self.logger.info(f"   Calculating cosine similarities...")
        
        matrix_norm = normalize(matrix, norm='l2', axis=1).astype(np.float32)
        nearest_similarities = np.zeros(len(pert_ids), dtype=np.float32)
        
        pert_ids_array = pert_ids.values
        unique_perts = np.unique(pert_ids_array)
        
        for pert_id in tqdm(unique_perts, desc="   Computing", leave=False):
            mask = pert_ids_array == pert_id
            indices = np.where(mask)[0]
            
            if len(indices) < 2:
                nearest_similarities[indices] = 0.0
                continue
            
            pert_data = matrix_norm[mask]
            sims = self._compute_max_similarities_jit(pert_data, indices, len(pert_ids))
            nearest_similarities[indices] = sims[indices]
        
        self.logger.info(f"   ✓ Mean similarity: {nearest_similarities.mean():.4f}")
        return nearest_similarities
    
    # ========== 过滤器方法 ==========
    
    def filter_1_dos_removal(
        self,
        matrix: np.ndarray,
        metadata: pd.DataFrame,
        remove_dos: bool = True
    ) -> Tuple[np.ndarray, pd.DataFrame, Dict]:
        """
        Filter 1: 移除DOS化合物（只保留trt_cp）
        
        参数:
            matrix: 表达矩阵
            metadata: 元数据
            remove_dos: 是否移除DOS
        
        返回:
            (过滤后的matrix, 过滤后的metadata, 统计信息)
        """
        filter_name = "filter_1_dos_removal"
        filter_params = {'remove_dos': remove_dos}
        
        self.logger.info(f"\n{'='*80}")
        self.logger.info(f"FILTER 1: DOS Removal")
        self.logger.info(f"{'='*80}")
        
        # 检查checkpoint
        ckpt_data = self.checkpoint_mgr.load_checkpoint(filter_name, filter_params)
        if ckpt_data is not None:
            return ckpt_data['matrix'], ckpt_data['metadata'], ckpt_data['stats']
        
        # 执行过滤
        initial_samples = len(metadata)
        initial_compounds = metadata['pert_id'].nunique()
        
        if remove_dos and 'pert_type' in metadata.columns:
            pert_type_counts = metadata['pert_type'].value_counts()
            self.logger.info(f"   pert_type distribution:")
            for ptype, count in pert_type_counts.items():
                self.logger.info(f"     - {ptype}: {count:,}")
            
            dos_mask = metadata['pert_type'] == 'trt_cp'
            n_removed = (~dos_mask).sum()
            
            matrix = matrix[dos_mask]
            metadata = metadata[dos_mask].reset_index(drop=True)
            
            self.logger.info(f"   ✓ Removed {n_removed:,} non-trt_cp samples")
        else:
            dos_mask = ~metadata['pert_id'].str.contains('DOS', case=False, na=False)
            n_removed = (~dos_mask).sum()
            
            matrix = matrix[dos_mask]
            metadata = metadata[dos_mask].reset_index(drop=True)
            
            self.logger.info(f"   ✓ Removed {n_removed:,} DOS samples")
        
        final_samples = len(metadata)
        final_compounds = metadata['pert_id'].nunique()
        
        self.logger.info(f"   Remaining: {final_samples:,} samples, {final_compounds:,} compounds")
        
        stats = {
            'initial_samples': initial_samples,
            'final_samples': final_samples,
            'removed_samples': n_removed,
            'initial_compounds': initial_compounds,
            'final_compounds': final_compounds
        }
        
        # 保存checkpoint
        data = {'matrix': matrix, 'metadata': metadata, 'stats': stats}
        self.checkpoint_mgr.save_checkpoint(filter_name, filter_params, data, stats)
        
        return matrix, metadata, stats
    
    def filter_2_min_observations(
        self,
        matrix: np.ndarray,
        metadata: pd.DataFrame,
        min_observations: int = 5
    ) -> Tuple[np.ndarray, pd.DataFrame, Dict]:
        """
        Filter 2: 移除观测数少于阈值的化合物
        
        参数:
            matrix: 表达矩阵
            metadata: 元数据
            min_observations: 最小观测数
        
        返回:
            (过滤后的matrix, 过滤后的metadata, 统计信息)
        """
        filter_name = "filter_2_min_observations"
        filter_params = {'min_observations': min_observations}
        
        self.logger.info(f"\n{'='*80}")
        self.logger.info(f"FILTER 2: Minimum Observations (≥{min_observations})")
        self.logger.info(f"{'='*80}")
        
        # 检查checkpoint
        ckpt_data = self.checkpoint_mgr.load_checkpoint(filter_name, filter_params)
        if ckpt_data is not None:
            return ckpt_data['matrix'], ckpt_data['metadata'], ckpt_data['stats']
        
        # 执行过滤
        initial_samples = len(metadata)
        initial_compounds = metadata['pert_id'].nunique()
        
        obs_counts = metadata.groupby('pert_id').size()
        valid_perts = obs_counts[obs_counts >= min_observations].index
        
        self.logger.info(f"   Compounds with ≥{min_observations} obs: {len(valid_perts):,}/{initial_compounds:,}")
        
        obs_mask = metadata['pert_id'].isin(valid_perts)
        
        matrix = matrix[obs_mask]
        metadata = metadata[obs_mask].reset_index(drop=True)
        
        final_samples = len(metadata)
        final_compounds = metadata['pert_id'].nunique()
        
        self.logger.info(f"   Remaining: {final_samples:,} samples, {final_compounds:,} compounds")
        
        stats = {
            'initial_samples': initial_samples,
            'final_samples': final_samples,
            'removed_samples': initial_samples - final_samples,
            'initial_compounds': initial_compounds,
            'final_compounds': final_compounds
        }
        
        # 保存checkpoint
        data = {'matrix': matrix, 'metadata': metadata, 'stats': stats}
        self.checkpoint_mgr.save_checkpoint(filter_name, filter_params, data, stats)
        
        gc.collect()
        return matrix, metadata, stats
    
    def filter_3_cosine_similarity(
        self,
        matrix: np.ndarray,
        metadata: pd.DataFrame,
        min_similarity: float = 0.12
    ) -> Tuple[np.ndarray, pd.DataFrame, Dict]:
        """
        Filter 3: 余弦相似度过滤
        
        参数:
            matrix: 表达矩阵
            metadata: 元数据
            min_similarity: 最小余弦相似度
        
        返回:
            (过滤后的matrix, 过滤后的metadata, 统计信息)
        """
        filter_name = "filter_3_cosine_similarity"
        filter_params = {'min_similarity': min_similarity}
        
        self.logger.info(f"\n{'='*80}")
        self.logger.info(f"FILTER 3: Cosine Similarity (≥{min_similarity})")
        self.logger.info(f"{'='*80}")
        
        # 检查checkpoint
        ckpt_data = self.checkpoint_mgr.load_checkpoint(filter_name, filter_params)
        if ckpt_data is not None:
            return ckpt_data['matrix'], ckpt_data['metadata'], ckpt_data['stats']
        
        # 执行过滤
        initial_samples = len(metadata)
        
        similarities = self.calculate_cosine_similarity(matrix, metadata['pert_id'])
        sim_mask = similarities >= min_similarity
        
        matrix = matrix[sim_mask]
        metadata = metadata[sim_mask].reset_index(drop=True)
        
        final_samples = len(metadata)
        n_removed = initial_samples - final_samples
        
        self.logger.info(f"   Removed: {n_removed:,} low-similarity samples")
        self.logger.info(f"   Remaining: {final_samples:,} samples")
        
        stats = {
            'initial_samples': initial_samples,
            'final_samples': final_samples,
            'removed_samples': n_removed,
            'mean_similarity': float(similarities[sim_mask].mean())
        }
        
        # 保存checkpoint
        data = {'matrix': matrix, 'metadata': metadata, 'stats': stats}
        self.checkpoint_mgr.save_checkpoint(filter_name, filter_params, data, stats)
        
        gc.collect()
        return matrix, metadata, stats
    
    def filter_4_dose_selection(
        self,
        matrix: np.ndarray,
        metadata: pd.DataFrame,
        dose_range: Tuple[float, float] = (1.0, 20.0),
        dose_bins: Optional[List[float]] = None,
        n_bins: int = 4
    ) -> Tuple[np.ndarray, pd.DataFrame, Dict]:
        """
        Filter 4: 剂量选择（基于直方图区间）
        
        对每个化合物：
        1. 将剂量分为多个区间（直方图）
        2. 选择样本数最多的区间
        3. 只保留该区间内的样本
        
        参数:
            matrix: 表达矩阵
            metadata: 元数据
            dose_range: 剂量范围（µM），例如 (1.0, 20.0)
            dose_bins: 自定义区间边界，例如 [1, 5, 10, 15, 20]
                    如果为 None，则在 dose_range 内均匀分为 n_bins 个区间
            n_bins: 区间数量（仅在 dose_bins=None 时生效），默认 4
        
        返回:
            (过滤后的matrix, 过滤后的metadata, 统计信息)
        
        示例:
            # 使用默认 4 个均匀区间：[1-5.75), [5.75-10.5), [10.5-15.25), [15.25-20]
            filter_4_dose_selection(matrix, metadata)
            
            # 自定义区间：[1-5), [5-10), [10-15), [15-20]
            filter_4_dose_selection(matrix, metadata, dose_bins=[1, 5, 10, 15, 20])
        """
        filter_name = "filter_4_dose_selection"
        filter_params = {
            'dose_range': dose_range,
            'dose_bins': dose_bins if dose_bins is not None else 'auto',
            'n_bins': n_bins
        }
        
        self.logger.info(f"\n{'='*80}")
        self.logger.info(f"FILTER 4: Dose Selection (Histogram-based)")
        self.logger.info(f"{'='*80}")
        self.logger.info(f"   Dose range: {dose_range[0]}-{dose_range[1]} µM")
        
        # 检查checkpoint
        ckpt_data = self.checkpoint_mgr.load_checkpoint(filter_name, filter_params)
        if ckpt_data is not None:
            return ckpt_data['matrix'], ckpt_data['metadata'], ckpt_data['stats']
        
        # 执行过滤
        if 'pert_dose' not in metadata.columns:
            self.logger.warning(f"   ⚠️  'pert_dose' not found, skipping")
            stats = {'skipped': True}
            return matrix, metadata, stats
        
        initial_samples = len(metadata)
        initial_compounds = metadata['pert_id'].nunique()
        
        # ========== Step 1: 解析剂量 ==========
        self.logger.info(f"\n   Step 1/4: Parsing dose values...")
        dose_str = metadata['pert_dose'].astype(str)
        metadata['dose_value'] = pd.to_numeric(
            dose_str.str.replace(r'[^\d.]', '', regex=True),
            errors='coerce'
        )
        metadata['dose_unit'] = dose_str.str.extract(r'([a-zA-Z]+)', expand=False).str.lower()
        
        # 转换单位到µM
        dose_value = metadata['dose_value'].values
        dose_unit = metadata['dose_unit'].values
        
        conditions = [
            dose_unit == 'nm',
            dose_unit == 'mm'
        ]
        choices = [
            dose_value / 1000,
            dose_value * 1000
        ]
        metadata['dose_uM'] = np.select(conditions, choices, default=dose_value)
        
        valid_dose_count = metadata['dose_uM'].notna().sum()
        self.logger.info(f"   ✓ Parsed {valid_dose_count:,} valid doses")
        
        # ========== Step 2: 应用剂量范围过滤 ==========
        self.logger.info(f"\n   Step 2/4: Filtering dose range...")
        dose_range_mask = (
            (metadata['dose_uM'] >= dose_range[0]) &
            (metadata['dose_uM'] <= dose_range[1]) &
            (~pd.isna(metadata['dose_uM']))
        )
        
        n_in_range = dose_range_mask.sum()
        n_out_range = len(metadata) - n_in_range
        self.logger.info(f"   ✓ Samples in range {dose_range[0]}-{dose_range[1]} µM: {n_in_range:,}")
        self.logger.info(f"   ✓ Samples outside range: {n_out_range:,}")
        
        matrix = matrix[dose_range_mask]
        metadata = metadata[dose_range_mask].reset_index(drop=True)
        
        # ========== Step 3: 定义剂量区间（bins）==========
        self.logger.info(f"\n   Step 3/4: Defining dose bins...")
        
        if dose_bins is None:
            # 自动生成均匀区间
            dose_bins = np.linspace(dose_range[0], dose_range[1], n_bins + 1)
            self.logger.info(f"   Using {n_bins} uniform bins:")
        else:
            dose_bins = np.array(dose_bins)
            n_bins = len(dose_bins) - 1
            self.logger.info(f"   Using {n_bins} custom bins:")
        
        # 打印区间信息
        for i in range(len(dose_bins) - 1):
            left = dose_bins[i]
            right = dose_bins[i + 1]
            bracket_right = ']' if i == len(dose_bins) - 2 else ')'
            self.logger.info(f"     Bin {i+1}: [{left:.2f}, {right:.2f}{bracket_right} µM")
        
        # 为每个样本分配区间
        metadata['dose_bin'] = pd.cut(
            metadata['dose_uM'],
            bins=dose_bins,
            labels=range(n_bins),
            include_lowest=True
        )
        
        # ========== Step 4: 为每个化合物选择样本最多的区间 ==========
        self.logger.info(f"\n   Step 4/4: Selecting modal bin for each compound...")
        
        def find_modal_bin(bin_series):
            """找到样本数最多的区间"""
            bin_counts = bin_series.value_counts()
            if len(bin_counts) == 0:
                return np.nan
            return bin_counts.idxmax()
        
        # 统计每个化合物在各区间的样本数
        modal_bins = metadata.groupby('pert_id')['dose_bin'].apply(find_modal_bin)
        
        # 打印区间选择统计
        self.logger.info(f"   Modal bin distribution across compounds:")
        modal_bin_dist = modal_bins.value_counts().sort_index()
        for bin_id, count in modal_bin_dist.items():
            if pd.notna(bin_id):
                left = dose_bins[int(bin_id)]
                right = dose_bins[int(bin_id) + 1]
                bracket_right = ']' if int(bin_id) == n_bins - 1 else ')'
                self.logger.info(
                    f"     Bin {int(bin_id)+1} [{left:.2f}, {right:.2f}{bracket_right}: "
                    f"{count:,} compounds ({count/len(modal_bins)*100:.1f}%)"
                )
        
        # 为每个样本添加其化合物的modal bin
        metadata['modal_bin'] = metadata['pert_id'].map(modal_bins)
        
        # 只保留modal bin内的样本
        modal_bin_mask = (
            (metadata['dose_bin'] == metadata['modal_bin']) &
            (~pd.isna(metadata['modal_bin']))
        )
        
        n_removed = len(metadata) - modal_bin_mask.sum()
        
        matrix = matrix[modal_bin_mask]
        metadata = metadata[modal_bin_mask].reset_index(drop=True)
        
        final_samples = len(metadata)
        final_compounds = metadata['pert_id'].nunique()
        
        self.logger.info(f"\n   ✅ Filter 4 Results:")
        self.logger.info(f"   ✓ Removed {n_removed:,} samples (non-modal bin)")
        self.logger.info(f"   ✓ Remaining: {final_samples:,} samples, {final_compounds:,} compounds")
        
        # ========== 统计信息 ==========
        stats = {
            'initial_samples': int(initial_samples),
            'final_samples': int(final_samples),
            'removed_samples': int(initial_samples - final_samples),
            'initial_compounds': int(initial_compounds),
            'final_compounds': int(final_compounds),
            'dose_range': dose_range,
            'dose_bins': dose_bins.tolist(),
            'n_bins': int(n_bins),
            'modal_bin_distribution': {
                int(k): int(v) for k, v in modal_bin_dist.items() if pd.notna(k)
            }
        }
        
        # ========== 可视化（可选）==========
        if False:  # 设置为 True 启用可视化
            self._plot_dose_histogram(metadata, dose_bins, modal_bins)
        
        # 清理临时列
        metadata.drop(
            ['dose_value', 'dose_unit', 'dose_bin', 'modal_bin'],
            axis=1,
            inplace=True,
            errors='ignore'
        )
        
        # 保存checkpoint
        data = {'matrix': matrix, 'metadata': metadata, 'stats': stats}
        self.checkpoint_mgr.save_checkpoint(filter_name, filter_params, data, stats)
        
        gc.collect()
        return matrix, metadata, stats


    def _plot_dose_histogram(
        self,
        metadata: pd.DataFrame,
        dose_bins: np.ndarray,
        modal_bins: pd.Series
    ):
        """
        可视化剂量分布直方图
        
        参数:
            metadata: 包含剂量信息的元数据
            dose_bins: 区间边界
            modal_bins: 每个化合物的modal bin
        """
        import matplotlib.pyplot as plt
        from pathlib import Path
        
        self.logger.info(f"\n   📊 Generating dose histogram visualization...")
        
        viz_dir = Path(self.data_dir) / "visualizations"
        viz_dir.mkdir(exist_ok=True, parents=True)
        
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        
        # ===== 子图1: 整体剂量分布 =====
        ax1 = axes[0, 0]
        ax1.hist(
            metadata['dose_uM'],
            bins=dose_bins,
            edgecolor='black',
            alpha=0.7,
            color='steelblue'
        )
        ax1.set_xlabel('Dose (µM)', fontsize=11)
        ax1.set_ylabel('Number of Samples', fontsize=11)
        ax1.set_title('Overall Dose Distribution', fontsize=12, fontweight='bold')
        ax1.grid(True, alpha=0.3)
        
        # 添加区间分隔线
        for bin_edge in dose_bins[1:-1]:
            ax1.axvline(bin_edge, color='red', linestyle='--', alpha=0.5, linewidth=1)
        
        # ===== 子图2: Modal bin分布（化合物层面）=====
        ax2 = axes[0, 1]
        modal_bin_counts = modal_bins.value_counts().sort_index()
        
        x_pos = range(len(modal_bin_counts))
        bars = ax2.bar(
            x_pos,
            modal_bin_counts.values,
            color='darkorange',
            edgecolor='black',
            alpha=0.8
        )
        
        # 设置x轴标签
        bin_labels = []
        for bin_id in modal_bin_counts.index:
            if pd.notna(bin_id):
                left = dose_bins[int(bin_id)]
                right = dose_bins[int(bin_id) + 1]
                bin_labels.append(f'[{left:.1f}, {right:.1f})')
            else:
                bin_labels.append('NaN')
        
        ax2.set_xticks(x_pos)
        ax2.set_xticklabels(bin_labels, rotation=45, ha='right')
        ax2.set_xlabel('Dose Bin (µM)', fontsize=11)
        ax2.set_ylabel('Number of Compounds', fontsize=11)
        ax2.set_title('Modal Bin Distribution\n(Compound Level)', fontsize=12, fontweight='bold')
        ax2.grid(axis='y', alpha=0.3)
        
        # 添加数值标签
        for bar, val in zip(bars, modal_bin_counts.values):
            ax2.text(
                bar.get_x() + bar.get_width()/2,
                bar.get_height(),
                f'{val:,}',
                ha='center',
                va='bottom',
                fontsize=9
            )
        
        # ===== 子图3: 随机选择10个化合物的剂量分布 =====
        ax3 = axes[1, 0]
        
        sample_compounds = np.random.choice(
            metadata['pert_id'].unique(),
            size=min(10, metadata['pert_id'].nunique()),
            replace=False
        )
        
        for i, pert_id in enumerate(sample_compounds):
            pert_data = metadata[metadata['pert_id'] == pert_id]['dose_uM']
            ax3.scatter(
                pert_data,
                [i] * len(pert_data),
                alpha=0.6,
                s=50,
                label=pert_id[:10]
            )
        
        ax3.set_xlabel('Dose (µM)', fontsize=11)
        ax3.set_ylabel('Compound Index', fontsize=11)
        ax3.set_title('Sample Dose Distribution\n(10 Random Compounds)', fontsize=12, fontweight='bold')
        ax3.grid(True, alpha=0.3)
        
        # 添加bin边界线
        for bin_edge in dose_bins:
            ax3.axvline(bin_edge, color='red', linestyle='--', alpha=0.3, linewidth=1)
        
        # ===== 子图4: 每个bin的样本数统计 =====
        ax4 = axes[1, 1]
        
        bin_sample_counts = metadata['dose_bin'].value_counts().sort_index()
        
        x_pos = range(len(bin_sample_counts))
        bars = ax4.bar(
            x_pos,
            bin_sample_counts.values,
            color='forestgreen',
            edgecolor='black',
            alpha=0.8
        )
        
        # 设置x轴标签
        bin_labels = []
        for bin_id in bin_sample_counts.index:
            left = dose_bins[int(bin_id)]
            right = dose_bins[int(bin_id) + 1]
            bin_labels.append(f'[{left:.1f}, {right:.1f})')
        
        ax4.set_xticks(x_pos)
        ax4.set_xticklabels(bin_labels, rotation=45, ha='right')
        ax4.set_xlabel('Dose Bin (µM)', fontsize=11)
        ax4.set_ylabel('Number of Samples', fontsize=11)
        ax4.set_title('Sample Distribution by Bin\n(Sample Level)', fontsize=12, fontweight='bold')
        ax4.grid(axis='y', alpha=0.3)
        
        # 添加数值标签
        for bar, val in zip(bars, bin_sample_counts.values):
            ax4.text(
                bar.get_x() + bar.get_width()/2,
                bar.get_height(),
                f'{val:,}',
                ha='center',
                va='bottom',
                fontsize=9
            )
        
        plt.suptitle(
            'Dose Selection Filter - Histogram-based Approach',
            fontsize=14,
            fontweight='bold',
            y=0.995
        )
        plt.tight_layout()
        
        output_path = viz_dir / "dose_histogram_filter.png"
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        self.logger.info(f"   ✓ Saved visualization: {output_path}")
        plt.close()
    
    def filter_5_timepoint_selection(
        self,
        matrix: np.ndarray,
        metadata: pd.DataFrame,
        valid_timepoints: List[int] = [6, 24]
    ) -> Tuple[np.ndarray, pd.DataFrame, Dict]:
        """
        Filter 5: 时间点选择
        
        参数:
            matrix: 表达矩阵
            metadata: 元数据
            valid_timepoints: 有效时间点（小时）
        
        返回:
            (过滤后的matrix, 过滤后的metadata, 统计信息)
        """
        filter_name = "filter_5_timepoint_selection"
        filter_params = {'valid_timepoints': valid_timepoints}
        
        self.logger.info(f"\n{'='*80}")
        self.logger.info(f"FILTER 5: Timepoint Selection ({valid_timepoints} hours)")
        self.logger.info(f"{'='*80}")
        
        # 检查checkpoint
        ckpt_data = self.checkpoint_mgr.load_checkpoint(filter_name, filter_params)
        if ckpt_data is not None:
            return ckpt_data['matrix'], ckpt_data['metadata'], ckpt_data['stats']
        
        # 执行过滤
        if 'pert_time' not in metadata.columns:
            self.logger.warning(f"   ⚠️  'pert_time' not found, skipping")
            stats = {'skipped': True}
            return matrix, metadata, stats
        
        initial_samples = len(metadata)
        
        # 转换为数值
        metadata['time_numeric'] = pd.to_numeric(metadata['pert_time'], errors='coerce')
        
        time_counts = metadata['time_numeric'].value_counts().head(10)
        self.logger.info(f"   Available timepoints:")
        for time_val, count in time_counts.items():
            self.logger.info(f"     - {time_val} hours: {count:,}")
        
        time_mask = metadata['time_numeric'].isin(valid_timepoints)
        
        matrix = matrix[time_mask]
        metadata = metadata[time_mask].reset_index(drop=True)
        
        final_samples = len(metadata)
        
        self.logger.info(f"   Removed: {initial_samples - final_samples:,} samples")
        self.logger.info(f"   Remaining: {final_samples:,} samples")
        
        stats = {
            'initial_samples': initial_samples,
            'final_samples': final_samples,
            'removed_samples': initial_samples - final_samples
        }
        
        # 保存checkpoint
        data = {'matrix': matrix, 'metadata': metadata, 'stats': stats}
        self.checkpoint_mgr.save_checkpoint(filter_name, filter_params, data, stats)
        
        gc.collect()
        return matrix, metadata, stats
    
    def filter_6_min_compounds_per_cell(
        self,
        matrix: np.ndarray,
        metadata: pd.DataFrame,
        min_compounds: int = 200,
        cell_col: str = 'cell_iname'
    ) -> Tuple[np.ndarray, pd.DataFrame, Dict]:
        """
        Filter 6: 移除化合物数少于阈值的细胞系
        
        参数:
            matrix: 表达矩阵
            metadata: 元数据
            min_compounds: 每个细胞系的最小化合物数
            cell_col: 细胞系ID列名
        
        返回:
            (过滤后的matrix, 过滤后的metadata, 统计信息)
        """
        filter_name = "filter_6_min_compounds_per_cell"
        filter_params = {
            'min_compounds': min_compounds,
            'cell_col': cell_col
        }
        
        self.logger.info(f"\n{'='*80}")
        self.logger.info(f"FILTER 6: Min Compounds Per Cell (≥{min_compounds})")
        self.logger.info(f"{'='*80}")
        
        # 检查checkpoint
        ckpt_data = self.checkpoint_mgr.load_checkpoint(filter_name, filter_params)
        if ckpt_data is not None:
            return ckpt_data['matrix'], ckpt_data['metadata'], ckpt_data['stats']
        
        # 执行过滤
        if cell_col not in metadata.columns:
            self.logger.warning(f"   ⚠️  '{cell_col}' not found, skipping")
            stats = {'skipped': True}
            return matrix, metadata, stats
        
        initial_samples = len(metadata)
        initial_cells = metadata[cell_col].nunique()
        
        compounds_per_cell = metadata.groupby(cell_col)['pert_id'].nunique()
        
        self.logger.info(f"   Cell lines before: {initial_cells:,}")
        self.logger.info(f"   Compounds per cell (mean): {compounds_per_cell.mean():.1f}")
        
        valid_cells = compounds_per_cell[compounds_per_cell >= min_compounds].index
        
        self.logger.info(f"   Cell lines with ≥{min_compounds} compounds: {len(valid_cells):,}")
        
        cell_mask = metadata[cell_col].isin(valid_cells)
        
        matrix = matrix[cell_mask]
        metadata = metadata[cell_mask].reset_index(drop=True)
        
        final_samples = len(metadata)
        final_cells = metadata[cell_col].nunique()
        
        self.logger.info(f"   Removed: {initial_samples - final_samples:,} samples from {initial_cells - final_cells:,} cell lines")
        self.logger.info(f"   Remaining: {final_samples:,} samples, {final_cells:,} cell lines")
        
        stats = {
            'initial_samples': initial_samples,
            'final_samples': final_samples,
            'removed_samples': initial_samples - final_samples,
            'initial_cells': initial_cells,
            'final_cells': final_cells
        }
        
        # 保存checkpoint
        data = {'matrix': matrix, 'metadata': metadata, 'stats': stats}
        self.checkpoint_mgr.save_checkpoint(filter_name, filter_params, data, stats)
        
        gc.collect()
        return matrix, metadata, stats
    
    def filter_7_cell_line_count(
        self,
        matrix: np.ndarray,
        metadata: pd.DataFrame,
        min_cell_lines: int = 5,
        max_cell_lines: int = 40,
        cell_col: str = 'cell_iname'
    ) -> Tuple[np.ndarray, pd.DataFrame, Dict]:
        """
        Filter 7: 细胞系数量过滤
        
        参数:
            matrix: 表达矩阵
            metadata: 元数据
            min_cell_lines: 最小细胞系数
            max_cell_lines: 最大细胞系数
            cell_col: 细胞系ID列名
        
        返回:
            (过滤后的matrix, 过滤后的metadata, 统计信息)
        """
        filter_name = "filter_7_cell_line_count"
        filter_params = {
            'min_cell_lines': min_cell_lines,
            'max_cell_lines': max_cell_lines,
            'cell_col': cell_col
        }
        
        self.logger.info(f"\n{'='*80}")
        self.logger.info(f"FILTER 7: Cell Line Count ({min_cell_lines}-{max_cell_lines})")
        self.logger.info(f"{'='*80}")
        
        # 检查checkpoint
        ckpt_data = self.checkpoint_mgr.load_checkpoint(filter_name, filter_params)
        if ckpt_data is not None:
            return ckpt_data['matrix'], ckpt_data['metadata'], ckpt_data['stats']
        
        # 执行过滤
        if cell_col not in metadata.columns:
            self.logger.warning(f"   ⚠️  '{cell_col}' not found, skipping")
            stats = {'skipped': True}
            return matrix, metadata, stats
        
        initial_samples = len(metadata)
        initial_compounds = metadata['pert_id'].nunique()
        
        cell_line_counts = metadata.groupby('pert_id')[cell_col].nunique()
        valid_perts = cell_line_counts[
            (cell_line_counts >= min_cell_lines) &
            (cell_line_counts <= max_cell_lines)
        ].index
        
        self.logger.info(f"   Compounds in {min_cell_lines}-{max_cell_lines} cell lines: {len(valid_perts):,}/{initial_compounds:,}")
        
        cell_mask = metadata['pert_id'].isin(valid_perts)
        
        matrix = matrix[cell_mask]
        metadata = metadata[cell_mask].reset_index(drop=True)
        
        final_samples = len(metadata)
        final_compounds = metadata['pert_id'].nunique()
        
        self.logger.info(f"   Removed: {initial_samples - final_samples:,} samples")
        self.logger.info(f"   Remaining: {final_samples:,} samples, {final_compounds:,} compounds")
        
        stats = {
            'initial_samples': initial_samples,
            'final_samples': final_samples,
            'removed_samples': initial_samples - final_samples,
            'initial_compounds': initial_compounds,
            'final_compounds': final_compounds
        }
        
        # 保存checkpoint
        data = {'matrix': matrix, 'metadata': metadata, 'stats': stats}
        self.checkpoint_mgr.save_checkpoint(filter_name, filter_params, data, stats)
        
        gc.collect()
        return matrix, metadata, stats
    
    # ========== 化学过滤器方法 ==========
    
    def _init_chemical_filters(self, nibr_csv_path: Optional[str] = None):
        """初始化化学过滤器"""
        if not RDKIT_AVAILABLE:
            self.logger.error("❌ RDKit not available, chemical filters disabled")
            return False
        
        # 初始化BRENK过滤器
        params = FilterCatalogParams()
        params.AddCatalog(FilterCatalogParams.FilterCatalogs.BRENK)
        self.brenk_catalog = FilterCatalog.FilterCatalog(params)
        
        # 初始化共价基团SMARTS
        self.covalent_smarts = [
            'C=CC(=O)', 'C=CC(=O)N', 'C=CC#N',
            '[C;!R](=O)Cl', '[C;!R](=O)O[C;!R](=O)', 'C(=O)N=[N+]=[N-]',
            'C1OC1', 'C1NC1',
            'ClCC(=O)N', 'BrCC(=O)N',
            '[CH;!R]=O',
            'N=C=O', 'N=C=S',
            'S(=O)(=O)Cl', 'S(=O)(=O)F',
            '[C;!R]#N',
            'OO',
            'C1(=O)NCC1',
            'C(=O)OC(=O)',
            'C1=CC(=O)C=CC1=O'
        ]
        
        # 初始化NIBR过滤器
        self.nibr_catalog = self._init_nibr_catalog(nibr_csv_path)
        
        return True
    
    def _init_nibr_catalog(self, csv_path: Optional[str] = None):
        """初始化NIBR过滤器目录"""
        if csv_path is None:
            csv_path = self.data_dir / "chem_filter" / "SubstructureFilter_HitTriaging_wPubChemExamples.csv"
        else:
            csv_path = Path(csv_path)
        
        if not csv_path.exists():
            self.logger.warning(f"⚠️  NIBR filter CSV not found: {csv_path}")
            return FilterCatalog.FilterCatalog()
        
        self.logger.info(f"   Loading NIBR filters from: {csv_path.name}")
        
        try:
            nibr_df = pd.read_csv(csv_path)
            self.logger.info(f"   ✓ Loaded {len(nibr_df)} NIBR patterns")
            
            nibr_catalog = FilterCatalog.FilterCatalog()
            n_added = 0
            
            for idx, row in nibr_df.iterrows():
                try:
                    pattern_name = row['PATTERN_NAME']
                    smarts = row['SMARTS']
                    min_count = 1 if row['MIN_COUNT'] == 0 else int(row['MIN_COUNT'])
                    severity = int(row['SEVERITY_SCORE'])
                    covalent = int(row['COVALENT'])
                    special_mol = int(row['SPECIAL_MOL'])
                    set_name = row['SET_NAME']
                    
                    filter_name = f"{pattern_name}_min({min_count})__{severity}__{covalent}__{special_mol}"
                    matcher = FilterCatalog.SmartsMatcher(filter_name, smarts, min_count)
                    
                    entry = FilterCatalog.FilterCatalogEntry(filter_name, matcher)
                    entry.SetProp('Scope', set_name)
                    entry.SetProp('Severity', str(severity))
                    nibr_catalog.AddEntry(entry)
                    
                    n_added += 1
                except:
                    continue
            
            self.logger.info(f"   ✓ Added {n_added} NIBR filters")
            return nibr_catalog
            
        except Exception as e:
            self.logger.error(f"   ❌ Error loading NIBR filters: {e}")
            return FilterCatalog.FilterCatalog()
    
    def filter_8_chemical_filters(
        self,
        matrix: np.ndarray,
        metadata: pd.DataFrame,
        min_mw: float = 60,
        max_mw: float = 1000,
        max_covalent_motifs: int = 1,
        max_nibr_flags: int = 9,
        nibr_csv_path: Optional[str] = None
    ) -> Tuple[np.ndarray, pd.DataFrame, Dict]:
        """
        Filter 8: 化学过滤器（整合所有化学过滤）
        
        参数:
            matrix: 表达矩阵
            metadata: 元数据
            min_mw: 最小分子量
            max_mw: 最大分子量
            max_covalent_motifs: 最大共价基团数
            max_nibr_flags: 最大NIBR标记数
            nibr_csv_path: NIBR过滤器CSV路径
        
        返回:
            (过滤后的matrix, 过滤后的metadata, 统计信息)
        """
        filter_name = "filter_8_chemical"
        filter_params = {
            'min_mw': min_mw,
            'max_mw': max_mw,
            'max_covalent_motifs': max_covalent_motifs,
            'max_nibr_flags': max_nibr_flags
        }
        
        self.logger.info(f"\n{'='*80}")
        self.logger.info(f"FILTER 8: Chemical Filters")
        self.logger.info(f"{'='*80}")
        
        # 检查checkpoint
        ckpt_data = self.checkpoint_mgr.load_checkpoint(filter_name, filter_params)
        if ckpt_data is not None:
            return ckpt_data['matrix'], ckpt_data['metadata'], ckpt_data['stats']
        
        # 检查RDKit
        if not RDKIT_AVAILABLE:
            self.logger.error("   ❌ RDKit not available, skipping chemical filters")
            stats = {'skipped': True, 'reason': 'RDKit not available'}
            return matrix, metadata, stats
        
        # 初始化化学过滤器
        if self.covalent_smarts is None:
            self._init_chemical_filters(nibr_csv_path)
        
        # 加载化合物信息
        if self.compound_info is None:
            self.load_compound_info()
        
        if self.compound_info is None:
            self.logger.error("   ❌ Compound info not available")
            stats = {'skipped': True, 'reason': 'Compound info not available'}
            return matrix, metadata, stats
        
        initial_samples = len(metadata)
        initial_compounds = metadata['pert_id'].nunique()
        
        # 确定SMILES列
        smiles_col = None
        for col in ['canonical_smiles', 'smiles', 'SMILES']:
            if col in self.compound_info.columns:
                smiles_col = col
                break
        
        if smiles_col is None:
            self.logger.error("   ❌ SMILES column not found")
            stats = {'skipped': True, 'reason': 'SMILES column not found'}
            return matrix, metadata, stats
        
        self.logger.info(f"   Using '{smiles_col}' for chemical structures")
        
        # 只处理训练数据中的化合物
        training_compounds = set(metadata['pert_id'].unique())
        compound_subset = self.compound_info[
            self.compound_info['pert_id'].isin(training_compounds)
        ].copy()
        
        self.logger.info(f"   Processing {len(compound_subset):,} compounds")
        
        # 应用各个化学过滤器
        passed_compounds = set()
        filter_results = {
            'mw': [],
            'covalent': [],
            'nibr': [],
            'brenk': []
        }
        
        # 预编译SMARTS
        smarts_mols = []
        for smarts in self.covalent_smarts:
            try:
                smarts_mol = Chem.MolFromSmarts(smarts)
                if smarts_mol:
                    smarts_mols.append(smarts_mol)
            except:
                pass
        
        self.logger.info(f"   Running chemical filters...")
        
        for idx, row in tqdm(compound_subset.iterrows(), total=len(compound_subset), desc="   Processing", leave=False):
            pert_id = row['pert_id']
            smiles = row[smiles_col]
            
            # 跳过无效SMILES
            if pd.isna(smiles) or smiles == '' or smiles == 'restricted':
                continue
            
            # 解析分子（禁用RDKit警告）
            mol = Chem.MolFromSmiles(str(smiles))
            if mol is None:
                continue
            
            # Filter 8.1: 分子量
            mw = Descriptors.MolWt(mol)
            if not (min_mw <= mw <= max_mw):
                filter_results['mw'].append(pert_id)
                continue
            
            # Filter 8.2: 共价基团
            motif_count = 0
            for smarts_mol in smarts_mols:
                if mol.HasSubstructMatch(smarts_mol):
                    motif_count += 1
            
            if motif_count > max_covalent_motifs:
                filter_results['covalent'].append(pert_id)
                continue
            
            # Filter 8.3: NIBR标记
            if self.nibr_catalog and self.nibr_catalog.GetNumEntries() > 0:
                matches = self.nibr_catalog.GetMatches(mol)
                
                if len(matches) > 0:
                    severity_scores = []
                    for entry in matches:
                        desc = entry.GetDescription()
                        parts = desc.split('__')
                        if len(parts) >= 2:
                            severity = int(parts[1])
                            severity_scores.append(severity)
                    
                    if 2 in severity_scores:
                        flag_count = 10
                    else:
                        flag_count = sum(1 for s in severity_scores if s == 1)
                    
                    if flag_count > max_nibr_flags:
                        filter_results['nibr'].append(pert_id)
                        continue
            
            # Filter 8.4: BRENK
            brenk_matches = self.brenk_catalog.GetMatches(mol)
            if len(brenk_matches) > 0:
                filter_results['brenk'].append(pert_id)
                continue
            
            # 通过所有过滤器
            passed_compounds.add(pert_id)
        
        # 应用过滤
        chem_mask = metadata['pert_id'].isin(passed_compounds)
        
        matrix = matrix[chem_mask]
        metadata = metadata[chem_mask].reset_index(drop=True)
        
        final_samples = len(metadata)
        final_compounds = metadata['pert_id'].nunique()
        
        self.logger.info(f"\n   Chemical Filter Results:")
        self.logger.info(f"     Passed all filters: {len(passed_compounds):,}/{len(training_compounds):,}")
        self.logger.info(f"     Failed MW filter: {len(filter_results['mw']):,}")
        self.logger.info(f"     Failed covalent filter: {len(filter_results['covalent']):,}")
        self.logger.info(f"     Failed NIBR filter: {len(filter_results['nibr']):,}")
        self.logger.info(f"     Failed BRENK filter: {len(filter_results['brenk']):,}")
        self.logger.info(f"\n   Removed: {initial_samples - final_samples:,} samples")
        self.logger.info(f"   Remaining: {final_samples:,} samples, {final_compounds:,} compounds")
        
        stats = {
            'initial_samples': initial_samples,
            'final_samples': final_samples,
            'removed_samples': initial_samples - final_samples,
            'initial_compounds': initial_compounds,
            'final_compounds': final_compounds,
            'passed_compounds': len(passed_compounds),
            'failed_mw': len(filter_results['mw']),
            'failed_covalent': len(filter_results['covalent']),
            'failed_nibr': len(filter_results['nibr']),
            'failed_brenk': len(filter_results['brenk'])
        }
        
        # 保存checkpoint
        data = {'matrix': matrix, 'metadata': metadata, 'stats': stats}
        self.checkpoint_mgr.save_checkpoint(filter_name, filter_params, data, stats)
        
        gc.collect()
        return matrix, metadata, stats
    
    # ========== 主流程方法 ==========
    
    def run_full_pipeline(
        self,
        # 数据质量控制参数
        remove_dos: bool = True,
        min_observations: int = 5,
        min_similarity: float = 0.12,
        dose_range: Tuple[float, float] = (1.0, 20.0),
        dose_bins: Optional[List[float]] = None,
        valid_timepoints: List[int] = [6, 24],
        min_compounds_per_cell: int = 200,
        min_cell_lines: int = 5,
        max_cell_lines: int = 40,
        # 化学过滤参数
        apply_chemical_filters: bool = True,
        min_mw: float = 60,
        max_mw: float = 1000,
        max_covalent_motifs: int = 1,
        max_nibr_flags: int = 9,
        nibr_csv_path: Optional[str] = None,
        # 输出参数
        output_path: Optional[str] = None
    ) -> Dict:
        """
        运行完整的过滤管道
        
        参数:
            # 数据质量控制
            remove_dos: 是否移除DOS化合物
            min_observations: 每个化合物的最小观测数
            min_similarity: 最小余弦相似度
            dose_range: 剂量范围（µM）
            standard_dose: 标准剂量（µM）
            valid_timepoints: 有效时间点（小时）
            min_compounds_per_cell: 每个细胞系的最小化合物数
            min_cell_lines: 每个化合物的最小细胞系数
            max_cell_lines: 每个化合物的最大细胞系数
            
            # 化学过滤
            apply_chemical_filters: 是否应用化学过滤器
            min_mw: 最小分子量
            max_mw: 最大分子量
            max_covalent_motifs: 最大共价基团数
            max_nibr_flags: 最大NIBR标记数
            nibr_csv_path: NIBR过滤器CSV路径
            
            # 输出
            output_path: 输出文件路径
        
        返回:
            训练数据字典
        """
        self.logger.info(f"\n{'='*80}")
        self.logger.info(f"🚀 Starting Full Filter Pipeline")
        self.logger.info(f"{'='*80}\n")
        
        # 加载元数据
        self.logger.info("Step 1: Loading metadata...")
        self.load_gene_info()
        self.load_cell_info()
        self.load_compound_info()
        
        # 加载Level 4数据
        self.logger.info("\nStep 2: Loading Level 4 signatures...")
        matrix, metadata, gene_meta = self.load_level4_signatures()
        
        initial_samples = len(matrix)
        initial_compounds = metadata['pert_id'].nunique()
        
        self.logger.info(f"\nInitial dataset:")
        self.logger.info(f"  Samples: {initial_samples:,}")
        self.logger.info(f"  Compounds: {initial_compounds:,}")
        
        # 应用过滤器
        self.logger.info("\nStep 3: Applying filters...")
        
        matrix, metadata, _ = self.filter_1_dos_removal(matrix, metadata, remove_dos)
        matrix, metadata, _ = self.filter_2_min_observations(matrix, metadata, min_observations)
        matrix, metadata, _ = self.filter_3_cosine_similarity(matrix, metadata, min_similarity)
        matrix, metadata, _ = self.filter_4_dose_selection(matrix, metadata, dose_range, dose_bins)
        matrix, metadata, _ = self.filter_5_timepoint_selection(matrix, metadata, valid_timepoints)
        matrix, metadata, _ = self.filter_6_min_compounds_per_cell(matrix, metadata, min_compounds_per_cell)
        matrix, metadata, _ = self.filter_7_cell_line_count(matrix, metadata, min_cell_lines, max_cell_lines)
        
        if apply_chemical_filters:
            matrix, metadata, _ = self.filter_8_chemical_filters(
                matrix, metadata, min_mw, max_mw, max_covalent_motifs, max_nibr_flags, nibr_csv_path
            )
        
        # 创建训练数据
        self.logger.info(f"\n{'='*80}")
        self.logger.info(f"✅ Filtering Complete")
        self.logger.info(f"{'='*80}")
        
        final_samples = len(matrix)
        final_compounds = metadata['pert_id'].nunique()
        final_cells = metadata['cell_iname'].nunique() if 'cell_iname' in metadata.columns else 'Unknown'
        
        self.logger.info(f"Final dataset:")
        self.logger.info(f"  Samples: {final_samples:,} ({final_samples/initial_samples*100:.1f}%)")
        self.logger.info(f"  Compounds: {final_compounds:,} ({final_compounds/initial_compounds*100:.1f}%)")
        self.logger.info(f"  Cell lines: {final_cells}")
        self.logger.info(f"  Gene features: {matrix.shape[1]}")
        
        # 创建标签和compound映射
        unique_perts = sorted(metadata['pert_id'].unique())
        pert_to_idx = {pert: idx for idx, pert in enumerate(unique_perts)}
        labels = np.array([pert_to_idx[p] for p in metadata['pert_id']], dtype=np.int32)
        
        # 获取基因名
        if self.gene_info is not None and hasattr(self, "landmark_col_indices"):
            landmark_mask = self.gene_info['feature_space'] == 'landmark'
            landmark_geneinfo = self.gene_info[landmark_mask]
            gene_names = list(landmark_geneinfo['gene_symbol'].values)
        else:
            gene_name_col = gene_meta.columns[0]
            gene_names = list(gene_meta[gene_name_col].values)
        
        # 创建3折划分
        self.logger.info(f"\nCreating 3-fold splits...")
        np.random.seed(42)
        folds = np.zeros(len(metadata), dtype=np.int32)
        
        for pert_id in metadata['pert_id'].unique():
            pert_mask = metadata['pert_id'] == pert_id
            pert_indices = np.where(pert_mask)[0]
            
            np.random.shuffle(pert_indices)
            n_samples = len(pert_indices)
            
            fold_sizes = [n_samples // 3] * 3
            for i in range(n_samples % 3):
                fold_sizes[i] += 1
            
            start_idx = 0
            for fold_id, size in enumerate(fold_sizes):
                end_idx = start_idx + size
                folds[pert_indices[start_idx:end_idx]] = fold_id
                start_idx = end_idx
        
        self.logger.info(f"Fold statistics:")
        for fold_id in range(3):
            fold_mask = folds == fold_id
            n_samples = fold_mask.sum()
            n_compounds = metadata[fold_mask]['pert_id'].nunique()
            self.logger.info(f"  Fold {fold_id}: {n_samples:,} samples, {n_compounds:,} compounds")
        
        # 构建训练数据字典
        training_data = {
            'X': matrix,
            'y': labels,
            'folds': folds,
            'sample_meta': metadata,
            'metadata': metadata,
            'gene_names': gene_names,
            'compound_names': list(unique_perts),
            'pert_to_idx': pert_to_idx
        }
        
        # 保存
        if output_path is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_path = self.data_dir / "processed_data" / f"training_data_{timestamp}.pkl"
        
        output_path = Path(output_path)
        output_path.parent.mkdir(parents=True, exist_ok=True)
        
        self.logger.info(f"\n💾 Saving to: {output_path}")
        with open(output_path, 'wb') as f:
            pickle.dump(training_data, f, protocol=4)
        
        file_size_mb = output_path.stat().st_size / (1024**2)
        self.logger.info(f"✓ Saved successfully ({file_size_mb:.1f} MB)")
        
        # 与论文对比
        self.logger.info(f"\n📊 Comparison with DrugReflector paper:")
        self.logger.info(f"  Paper: 425,242 obs, 9,597 compounds, 52 cells")
        self.logger.info(f"  Ours:  {final_samples:,} obs, {final_compounds:,} compounds, {final_cells} cells")
        
        self.logger.info(f"\n{'='*80}")
        self.logger.info(f"✅ Pipeline Complete!")
        self.logger.info(f"{'='*80}")
        
        return training_data


def main():
    """主程序"""
    # 配置路径
    data_dir = "D:/科研/Models/drugreflector/datasets/LINCS2020"
    output_dir = "D:/科研/Models/drugreflector/processed_data"
    
    # 创建过滤管道
    pipeline = LINCS2020FilterPipeline(
        data_dir=data_dir,
        cache_dir=Path(data_dir) / "filter_cache",
        overwrite=True  # 设置为False可以使用缓存
    )
    
    # 运行完整管道
    try:
        training_data = pipeline.run_full_pipeline(
            # 数据质量控制
            remove_dos=True,
            min_observations=5,
            min_similarity=0.12,
            dose_range=(1.0, 20.0),
            dose_bins=[1.0, 20.0],
            valid_timepoints=[6, 24],
            min_compounds_per_cell=270,
            min_cell_lines=5,
            max_cell_lines=200,
            # 化学过滤
            apply_chemical_filters=True,
            min_mw=60,
            max_mw=1000,
            max_covalent_motifs=1,
            max_nibr_flags=9,
            nibr_csv_path=None,  # 自动查找
            # 输出
            output_path=Path(output_dir) / "training_data_unified_pipeline.pkl"
        )
        
        print(f"\n🎯 Ready for training!")
        return training_data
        
    except Exception as e:
        print(f"\n{'='*80}")
        print(f"❌ ERROR")
        print(f"{'='*80}")
        print(f"Type: {type(e).__name__}")
        print(f"Message: {e}")
        
        import traceback
        traceback.print_exc()
        
        return None


if __name__ == "__main__":
    training_data = main()

🔬 LINCS 2020 Unified Filter Pipeline
Data directory: D:\科研\Models\drugreflector\datasets\LINCS2020
Cache directory: D:\科研\Models\drugreflector\datasets\LINCS2020\filter_cache
Overwrite mode: ON


🚀 Starting Full Filter Pipeline

Step 1: Loading metadata...
📖 Loading gene information...
   ✓ Loaded 12,328 genes
   ✓ Landmark genes: 978
📖 Loading cell information...
   ✓ Loaded 240 cell lines
📖 Loading compound information...
   ✓ Loaded 39,321 compounds

Step 2: Loading Level 4 signatures...

📖 Loading Level 4 Signatures
📖 Reading GCTX file: level4_beta_trt_cp_n1805898x12328.gctx
   File size: 82.94 GB
   Matrix shape: (1805898, 12328)
   ⚠️  Detected ROW/COL swap, correcting...
   Using 978 landmark genes
   Loading data...
   ✓ Loaded matrix: (1805898, 978)
   ✓ Memory: 6.58 GB
📖 Loading instance information...
   ✓ Loaded 3,026,460 instances

Initial dataset:
  Samples: 1,805,898
  Compounds: 34,419

Step 3: Applying filters...

FILTER 1: DOS Removal
   ⚠️  Overwrite mode enabled, ig

In [2]:
"""
为已处理的数据添加 Level 3 表达值
- X: Level 4 Z-score (已有)
- X_ctx: Level 3 control baseline (已有)
- X_post: Level 3 treatment 原始表达值 (新增)
"""

import numpy as np
import pandas as pd
import pickle
import h5py
import gc
from pathlib import Path
from tqdm import tqdm

# ================= 配置路径 =================
DATA_DIR = Path(r"D:/科研/Models/drugreflector/datasets/LINCS2020")
PROCESSED_DIR = Path(r"D:/科研/Models/drugreflector/processed_data")

# 输入：已包含 X_ctx 的数据
INPUT_FILE = PROCESSED_DIR / "training_data_lincs2020_chemfiltered_1201_l_dual_head.pkl"

# 输出：添加 X_post 后的数据
OUTPUT_FILE = PROCESSED_DIR / "training_data_lincs2020_chemfiltered_1201_l_triple_head.pkl"

# Level 3 treatment GCTX 文件
LEVEL3_TREATMENT_FILE = DATA_DIR / "level3_beta_trt_cp_n1805898x12328.gctx"


def load_landmark_indices():
    """加载 landmark 基因的列索引"""
    gene_path = DATA_DIR / "geneinfo_beta.txt"
    gene_info = pd.read_csv(gene_path, sep='\t')
    landmark_indices = np.where(gene_info['feature_space'] == 'landmark')[0]
    print(f"   ✓ Landmark genes: {len(landmark_indices)}")
    return sorted(landmark_indices)


def read_level3_gctx_metadata(gctx_path):
    """读取 GCTX 的样本元数据（sample_id）"""
    print(f"\n📖 Reading GCTX metadata from: {gctx_path.name}...")
    
    with h5py.File(gctx_path, 'r') as f:
        # 检查数据结构
        if '/0/META/ROW/id' in f:
            # 标准 LINCS GCTX 结构：ROW=samples, COL=genes
            sample_ids = f['/0/META/ROW/id'][()].astype(str)
            data_shape = f['/0/DATA/0/matrix'].shape
            print(f"   ✓ Format: Samples x Genes")
            print(f"   ✓ Matrix shape: {data_shape}")
            is_transposed = False
        elif '/0/META/COL/id' in f:
            # 转置格式：COL=samples, ROW=genes
            sample_ids = f['/0/META/COL/id'][()].astype(str)
            data_shape = f['/0/DATA/0/matrix'].shape
            print(f"   ✓ Format: Genes x Samples (transposed)")
            print(f"   ✓ Matrix shape: {data_shape}")
            is_transposed = True
        else:
            raise ValueError("Cannot find sample IDs in GCTX file")
    
    return sample_ids, is_transposed


def read_level3_treatment_data(
    gctx_path,
    target_sample_ids,
    landmark_indices,
    is_transposed=False,
    chunk_size=50000
):
    """
    从 Level 3 GCTX 中读取指定样本的 landmark 基因表达值
    
    参数:
        gctx_path: GCTX 文件路径
        target_sample_ids: 需要提取的样本ID列表（按顺序）
        landmark_indices: landmark 基因的列索引
        is_transposed: 数据是否为 Genes x Samples 格式
        chunk_size: 分块读取大小（样本数）
    
    返回:
        X_post: (N_samples, N_genes) 的 Level 3 表达矩阵
    """
    print(f"\n⚡ Extracting Level 3 expression data...")
    print(f"   Target samples: {len(target_sample_ids):,}")
    print(f"   Landmark genes: {len(landmark_indices)}")
    
    with h5py.File(gctx_path, 'r') as f:
        # 读取所有样本ID
        if is_transposed:
            all_sample_ids = f['/0/META/COL/id'][()].astype(str)
        else:
            all_sample_ids = f['/0/META/ROW/id'][()].astype(str)
        
        # 创建样本ID到索引的映射
        print(f"   Creating sample ID index...")
        sample_id_to_idx = {sid: idx for idx, sid in enumerate(all_sample_ids)}
        
        # 找到目标样本在 GCTX 中的行索引
        target_indices = []
        missing_samples = []
        
        for sid in tqdm(target_sample_ids, desc="   Mapping sample IDs"):
            if sid in sample_id_to_idx:
                target_indices.append(sample_id_to_idx[sid])
            else:
                target_indices.append(-1)  # 标记为缺失
                missing_samples.append(sid)
        
        target_indices = np.array(target_indices)
        
        if missing_samples:
            print(f"   ⚠️  Warning: {len(missing_samples):,} samples not found in Level 3 data")
            print(f"      (Will fill with zeros)")
        
        # 准备输出矩阵
        n_samples = len(target_sample_ids)
        n_genes = len(landmark_indices)
        X_post = np.zeros((n_samples, n_genes), dtype=np.float32)
        
        # 访问数据集
        ds = f['/0/DATA/0/matrix']
        
        if is_transposed:
            # 格式: Genes x Samples
            print(f"   Reading transposed data (Genes x Samples)...")
            
            # 先读取所有 landmark 基因的行
            print(f"   Loading landmark genes...")
            landmark_data = ds[landmark_indices, :].astype(np.float32)  # (N_genes, All_samples)
            
            # 然后按列索引提取目标样本
            print(f"   Extracting target samples...")
            valid_mask = target_indices >= 0
            valid_indices = target_indices[valid_mask]
            
            X_post[valid_mask] = landmark_data[:, valid_indices].T  # 转置为 (N_samples, N_genes)
            
        else:
            # 格式: Samples x Genes
            print(f"   Reading data (Samples x Genes)...")
            
            # 分批读取样本
            valid_mask = target_indices >= 0
            valid_indices = target_indices[valid_mask]
            valid_positions = np.where(valid_mask)[0]
            
            # 按 chunk 读取
            n_valid = len(valid_indices)
            n_chunks = (n_valid + chunk_size - 1) // chunk_size
            
            for chunk_idx in tqdm(range(n_chunks), desc="   Reading chunks"):
                start_idx = chunk_idx * chunk_size
                end_idx = min(start_idx + chunk_size, n_valid)
                
                chunk_gctx_indices = valid_indices[start_idx:end_idx]
                chunk_output_positions = valid_positions[start_idx:end_idx]
                
                # 读取这批样本的 landmark 基因
                # 注意：对于不连续的行索引，需要逐行读取或使用 fancy indexing
                chunk_data = ds[chunk_gctx_indices][:, landmark_indices].astype(np.float32)
                
                X_post[chunk_output_positions] = chunk_data
        
        print(f"   ✓ Extracted Level 3 data: {X_post.shape}")
        print(f"   ✓ Data range: [{X_post.min():.3f}, {X_post.max():.3f}]")
        print(f"   ✓ Mean: {X_post.mean():.3f}, Std: {X_post.std():.3f}")
        
        # 统计非零样本
        non_zero_samples = (X_post.sum(axis=1) != 0).sum()
        print(f"   ✓ Non-zero samples: {non_zero_samples:,} / {n_samples:,}")
        
    return X_post


def main():
    print(f"{'='*80}")
    print(f"🧬 Adding Level 3 Expression Data (X_post)")
    print(f"{'='*80}")
    
    # ========== 1. 加载已处理的数据 ==========
    print(f"\n📖 Loading input data: {INPUT_FILE.name}...")
    with open(INPUT_FILE, 'rb') as f:
        data = pickle.load(f)
    
    print(f"   ✓ Current keys: {list(data.keys())}")
    print(f"   ✓ Samples: {len(data['X']):,}")
    print(f"   ✓ Genes: {data['X'].shape[1]}")
    
    # 检查是否已有 X_ctx
    if 'X_ctx' in data:
        print(f"   ✓ X_ctx already exists: {data['X_ctx'].shape}")
    else:
        print(f"   ⚠️  X_ctx not found. Please run the baseline generation script first.")
        return
    
    # ========== 2. 获取样本ID列表 ==========
    sample_meta = data['sample_meta']
    
    # 确定样本ID列名
    if 'sample_id' in sample_meta.columns:
        sample_id_col = 'sample_id'
    elif 'id' in sample_meta.columns:
        sample_id_col = 'id'
    else:
        raise ValueError("Cannot find sample_id column in metadata")
    
    target_sample_ids = sample_meta[sample_id_col].values
    print(f"\n   Using column '{sample_id_col}' for sample matching")
    print(f"   Target samples: {len(target_sample_ids):,}")
    
    # ========== 3. 加载 landmark 基因索引 ==========
    landmark_indices = load_landmark_indices()
    
    # ========== 4. 读取 Level 3 元数据 ==========
    _, is_transposed = read_level3_gctx_metadata(LEVEL3_TREATMENT_FILE)
    
    # ========== 5. 提取 Level 3 表达数据 ==========
    X_post = read_level3_treatment_data(
        LEVEL3_TREATMENT_FILE,
        target_sample_ids,
        landmark_indices,
        is_transposed=is_transposed,
        chunk_size=50000
    )
    
    # ========== 6. 添加到数据字典并保存 ==========
    data['X_post'] = X_post
    
    print(f"\n{'='*80}")
    print(f"📊 Final Dataset Summary")
    print(f"{'='*80}")
    print(f"  X (Level 4 Z-score):       {data['X'].shape}, dtype={data['X'].dtype}")
    if 'X_ctx' in data:
        print(f"  X_ctx (Level 3 baseline):  {data['X_ctx'].shape}, dtype={data['X_ctx'].dtype}")
    print(f"  X_post (Level 3 treatment): {data['X_post'].shape}, dtype={data['X_post'].dtype}")
    print(f"  Samples: {len(sample_meta):,}")
    print(f"  Compounds: {len(data['compound_names']):,}")
    
    # 保存
    print(f"\n💾 Saving to: {OUTPUT_FILE}...")
    OUTPUT_FILE.parent.mkdir(parents=True, exist_ok=True)
    
    with open(OUTPUT_FILE, 'wb') as f:
        pickle.dump(data, f, protocol=4)
    
    file_size_mb = OUTPUT_FILE.stat().st_size / (1024**2)
    print(f"   ✓ Saved successfully ({file_size_mb:.1f} MB)")
    
    print(f"\n{'='*80}")
    print(f"✅ Complete!")
    print(f"{'='*80}")
    print(f"📁 Output file: {OUTPUT_FILE}")
    print(f"\n🎯 Dataset now contains:")
    print(f"   • X: Level 4 Z-scores (for main prediction)")
    print(f"   • X_ctx: Level 3 control baseline (for context)")
    print(f"   • X_post: Level 3 treatment expression (for reconstruction)")


if __name__ == "__main__":
    main()

🧬 Adding Level 3 Expression Data (X_post)

📖 Loading input data: training_data_lincs2020_chemfiltered_1201_l_dual_head.pkl...
   ✓ Current keys: ['X', 'y', 'folds', 'sample_meta', 'metadata', 'gene_names', 'compound_names', 'pert_to_idx', 'X_ctx', 'X_ctx_embeddings']
   ✓ Samples: 509,006
   ✓ Genes: 978
   ✓ X_ctx already exists: (509006, 978)

   Using column 'sample_id' for sample matching
   Target samples: 509,006
   ✓ Landmark genes: 978

📖 Reading GCTX metadata from: level3_beta_trt_cp_n1805898x12328.gctx...


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'D:\科研\Models\drugreflector\datasets\LINCS2020\level3_beta_trt_cp_n1805898x12328.gctx', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)